**Competition link:**
> [https://zindi.africa/competitions/indabax-tunisia-2023](https://zindi.africa/competitions/indabax-tunisia-2023)

**Describtion:**
> This competition aims to develop a model that predicts protein stability when a single amino acid mutation occurs. Participants contribute to understanding mutation-related diseases and the impact of stability loss on protein function. The model analyzes wild-type and mutant protein sequences, focusing on predicting folding stability differences caused by point mutations.


**Summary:**
> I used the the difference of the Embeddings' values to train an xgboost model and predict the ddg (Delta Delta G is a metric for predicting how a single point mutation wil affect protein stability)
then spent time fine-tuning the model and experimenting with different parameters to achieve the best result.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#@title Download data from GCP bucket
import sys

if 'google.colab' in sys.modules:
  !gsutil -m cp -r gs://indaba-data .
else:
  !mkdir -p indaba-data/train
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train.csv --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_mut.pt --continue
  !wget -P indaba-data/train https://storage.googleapis.com/indaba-data/train/train_wt.pt --continue

  !mkdir -p indaba-data/test
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test.csv --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_mut.pt --continue
  !wget -P indaba-data/test https://storage.googleapis.com/indaba-data/test/test_wt.pt --continue

--2023-05-14 08:37:26--  https://storage.googleapis.com/indaba-data/train/train.csv
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 142.250.1.128, 142.250.103.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53721771 (51M) [text/csv]
Saving to: ‘indaba-data/train/train.csv’

train.csv           100%[===================>]  51.23M   190MB/s    in 0.3s    

2023-05-14 08:37:27 (190 MB/s) - ‘indaba-data/train/train.csv’ saved [53721771/53721771]

--2023-05-14 08:37:28--  https://storage.googleapis.com/indaba-data/train/train_mut.pt
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.125.128, 142.250.136.128, 142.250.148.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1739664107 (1.6G) [application/octet-stream]
Saving to: 

Importing The Necessary Dependencies

In [ ]:
import pandas as pd
import torch
import numpy as np
%cd indaba-data

/kaggle/working/indaba-data


Converting torch tensors to numpy arrays

In [ ]:
# Load Embedding tensors & Traing csv
# Embeddings were calculated using the ESM 650M pretrained model
# Tensor shape of embedded data:  [data_len,1280]
# There are no sequences in the Embedding tensors as we've performed an average of it (torch.mean(embed, dim=1))
# More details in https://huggingface.co/facebook/esm2_t33_650M_UR50D

wt_emb = torch.load("train/train_wt.pt").detach().cpu().numpy()
mut_emb = torch.load("train/train_mut.pt").detach().cpu().numpy()

wt_emb_t = torch.load("test/test_wt.pt").detach().cpu().numpy()
mut_emb_t = torch.load("test/test_mut.pt").detach().cpu().numpy()

df = pd.read_csv("train/train.csv")
df_t = pd.read_csv("test/test.csv")

In [ ]:
XXX = wt_emb - mut_emb
XXX_t = wt_emb_t - mut_emb_t

In [ ]:
XXX

array([[-0.0701555 , -0.00962102,  0.00554383, ...,  0.11682439,
        -0.12814403, -0.04074726],
       [-0.10550089, -0.0528065 ,  0.05350137, ..., -0.04076052,
        -0.068295  ,  0.00588027],
       [-0.09242268, -0.04318464, -0.07546735, ...,  0.06495333,
        -0.10553503, -0.00090995],
       ...,
       [-0.10059148, -0.02686656,  0.03313243, ...,  0.0278033 ,
         0.01464283, -0.01031012],
       [-0.0187971 , -0.02095222, -0.04726124, ...,  0.06330132,
         0.05002809, -0.08174077],
       [-0.12369898, -0.10786104, -0.02126551, ...,  0.08931351,
        -0.02440596, -0.06123579]], dtype=float32)

In [ ]:
import xgboost as xg
from sklearn.metrics import mean_squared_error as MSE

y = df['ddg'].to_numpy()
X = XXX

xgb_r_4 = xg.XGBRegressor(booster="gblinear",
                          objective ='reg:linear',
                          learning_rate=0.001,
                          n_estimators=70,
                          seed = 123)
xgb_r_4.fit(X, y)

[06:49:17] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.001, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=50, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [ ]:
pred = xgb_r_4.predict(XXX_t)

In [ ]:
y = df['ddg'].to_numpy()
X = XXX

xgb_r_v3 = xg.XGBRegressor(booster="gblinear", objective ='reg:linear',learning_rate=0.001, n_estimators=150, seed = 123)

xgb_r_v3.fit(X, y)

[10:29:52] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(base_score=None, booster='gblinear', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.001, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=150, n_jobs=None, num_parallel_tree=None,
             objective='reg:linear', predictor=None, ...)

In [ ]:
import pickle
pickle.dump(xgb_r_5, open('xgb_v1.sav', 'wb'))

In [ ]:
pred=xgb_r_v3.predict(XXX_t)

In [ ]:
result=pd.DataFrame({"ID":df_t["ID"].to_numpy(),"ddg":pred})
result.to_csv("sub8.csv", index=False)